### create data

In [1]:
"""Capacited Vehicles Routing Problem (CVRP)."""

from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

In [2]:

def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = [
        [
          0,75,58,45,56,34,50,59,54,59,23,70,90,41,40,78,21,33,56,34,71
        ],
        [
           75,0,33,57,72,50,18,25,42,10,41,29,36,72,30,71,17,71,55,67,39
        ],
        [
            58,33,0,47,63,42,46,16,12,42,42,70,56,31,68,68,28,69,57,41,24
        ],
        [
            45,57,47,0,20,40,34,60,23,23,46,62,65,34,61,42,53,49,27,65,61
        ],
        [
           56,72,63,20,0,86,80,34,31,87,70,52,68,66,59,85,88,86,88,64,35 
        ],
        [
           34,50,42,40,86,0,64,34,73,66,15,70,29,67,51,37,10,24,31,77,51
        ],
        [
            50,18,46,34,80,64,0,54,88,47,74,87,76,76,80,67,39,58,62,63,54
        ],
        [
            59,25,16,60,34,34,54,0,49,65,57,46,50,48,77,50,52,66,46,48,61
        ],
        [
            54,42,12,23,31,73,88,49,0,37,19,46,74,81,23,46,68,66,59,88,66
        ],
        [
            59,10,42,23,87,66,47,65,37,0,69,28,48,25,64,39,42,78,45,32,45
        ],
        [
            23,41,42,46,70,15,74,57,19,69,0,43,34,45,46,44,37,51,44,39,35
        ],
        [
            70,29,70,62,52,70,87,46,46,28,43,0,25,32,43,28,23,34,47,42,32
        ],
        [
            90,36,56,65,68,29,76,50,74,48,34,25,0,12,21,41,11,42,22,11,38
        ],
        [
            41,72,31,34,66,67,76,48,81,25,45,32,12,0,58,48,52,44,59,44,54
        ],
        [
           40,30,68,61,59,51,80,77,23,64,46,43,21,58,0,43,17,43,42,7,18
        ],
        [
           78,71,68,42,85,37,67,50,46,39,44,28,41,48,43,0,68,46,61,68,46
        ],
        [
            21,17,28,53,88,10,39,52,68,42,37,23,11,52,17,68,0,41,62,49,61
        ],
        [
            33,71,69,49,86,24,58,66,66,78,51,34,42,44,43,46,41,0,44,23,13
        ],
        [
            56,55,57,27,88,31,62,46,59,45,44,47,22,59,42,61,62,44,0,45,22
        ],
        [
            34,67,41,65,64,77,63,48,88,32,39,42,11,44,77,68,49,23,45,0,10
        ],
        [
            71,39,24,61,35,51,54,61,66,45,35,32,38,54,18,46,61,13,22,10,0
        ],
    ]
    data['demands'] = [0, 1, 1, 2, 4, 2, 4, 8, 8, 1, 2, 1, 2, 4, 4, 8, 8,6,7,4,7]
    data['vehicle_capacities'] = [30,30,30]
    data['num_vehicles'] = 3
    data['depot'] = 0
    return data

In [3]:
def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f'Objective: {solution.ObjectiveValue()}')
    total_distance = 0
    total_load = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data['demands'][node_index]
            plan_output += ' {0} Load({1}) -> '.format(node_index, route_load)
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += ' {0} Load({1})\n'.format(manager.IndexToNode(index),
                                                 route_load)
        plan_output += 'Distance of the route: {}km\n'.format(route_distance)
        plan_output += 'Load of the route: {}\n'.format(route_load)
        print(plan_output)
        total_distance += route_distance
        total_load += route_load
    print('Total distance of all routes: {}km'.format(total_distance))
    print('Total load of all routes: {}'.format(total_load))


In [4]:
def main():
    """Solve the CVRP problem."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

 # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)


    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(
        demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
    search_parameters.time_limit.FromSeconds(1)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)


if __name__ == '__main__':
    main()

Objective: 565
Route for vehicle 0:
 0 Load(0) ->  4 Load(4) ->  3 Load(6) ->  6 Load(10) ->  1 Load(11) ->  9 Load(12) ->  15 Load(20) ->  11 Load(21) ->  12 Load(23) ->  13 Load(27) ->  0 Load(27)
Distance of the route: 283km
Load of the route: 27

Route for vehicle 1:
 0 Load(0) ->  16 Load(8) ->  14 Load(12) ->  19 Load(16) ->  20 Load(23) ->  17 Load(29) ->  0 Load(29)
Distance of the route: 101km
Load of the route: 29

Route for vehicle 2:
 0 Load(0) ->  10 Load(2) ->  8 Load(10) ->  2 Load(11) ->  7 Load(19) ->  18 Load(26) ->  5 Load(28) ->  0 Load(28)
Distance of the route: 181km
Load of the route: 28

Total distance of all routes: 565km
Total load of all routes: 84
